In [12]:
import pandas as pd
import numpy as np

In [2]:
### PARAMETERS ###
PERIOD_AHEAD = 10

In [3]:
DATA = (
    pd.read_csv('../data/SP500_TRI.csv')
    .loc[:,['Date', 'Close']]
)

DATA['Date'] = (
    pd.to_datetime(DATA['Date'], format='%d-%m-%Y')
)

display(DATA)

,Date,Close
0,1988-01-04,256.019989
1,1988-01-05,258.769989
2,1988-01-06,259.029999
3,1988-01-07,261.209991
4,1988-01-08,243.550003
...,...,...
9312,2024-12-17,13277.549805
9313,2024-12-18,12886.209961
9314,2024-12-19,12875.089844
9315,2024-12-20,13015.660156


In [4]:
DATA['Date'].dt.to_period('M')

0       1988-01
1       1988-01
2       1988-01
3       1988-01
4       1988-01
         ...   
9312    2024-12
9313    2024-12
9314    2024-12
9315    2024-12
9316    2024-12
Name: Date, Length: 9317, dtype: period[M]

In [5]:
series_working_dates = DATA.copy()

series_working_dates['Year_Month'] =(
    series_working_dates['Date'].dt.to_period('M')
)

series_working_dates = (
    series_working_dates
    .groupby(['Year_Month'])
    .cumcount()
    # .groupby(0)
    # .count()
)

display(series_working_dates)

0        0
1        1
2        2
3        3
4        4
        ..
9312    11
9313    12
9314    13
9315    14
9316    15
Length: 9317, dtype: int64

In [6]:
data_working_day = (
    DATA
    .copy()
)

data_working_day['Num_Workday'] = series_working_dates

display(data_working_day)

,Date,Close,Num_Workday
0,1988-01-04,256.019989,0
1,1988-01-05,258.769989,1
2,1988-01-06,259.029999,2
3,1988-01-07,261.209991,3
4,1988-01-08,243.550003,4
...,...,...,...
9312,2024-12-17,13277.549805,11
9313,2024-12-18,12886.209961,12
9314,2024-12-19,12875.089844,13
9315,2024-12-20,13015.660156,14


In [17]:
data_returns = (
    data_working_day.copy()
)

data_returns['Test_Period_Ahead_Return'] = (
    data_returns['Close'].shift(-PERIOD_AHEAD)
)

data_returns['Single_Period_Ahead_Return'] = (
    data_returns['Close'].shift(-1)
)

data_returns['Test_Period_Ahead_Return'] = (
    (data_returns['Test_Period_Ahead_Return'] - data_returns['Close']) / np.abs(data_returns['Close'])
)

data_returns['Single_Period_Ahead_Return'] = (
    (data_returns['Single_Period_Ahead_Return'] - data_returns['Close']) / np.abs(data_returns['Close'])
)

display(data_returns)

,Date,Close,Num_Workday,Test_Period_Ahead_Return,Single_Period_Ahead_Return
0,1988-01-04,256.019989,0,-0.015233,0.010741
1,1988-01-05,258.769989,1,-0.035553,0.001005
2,1988-01-06,259.029999,2,-0.062387,0.008416
3,1988-01-07,261.209991,3,-0.068259,-0.067608
4,1988-01-08,243.550003,4,0.013139,0.016916
...,...,...,...,...,...
9312,2024-12-17,13277.549805,11,NaN,-0.029474
9313,2024-12-18,12886.209961,12,NaN,-0.000863
9314,2024-12-19,12875.089844,13,NaN,0.010918
9315,2024-12-20,13015.660156,14,NaN,0.007348


In [33]:
data_volatility = (
    data_returns.copy()
)

data_volatility['Test_Period_Ahead_Std'] = (
    data_volatility['Single_Period_Ahead_Return']
    .shift(-PERIOD_AHEAD)
    .rolling(window=PERIOD_AHEAD)
    .apply(np.var)
)

data_volatility = (
    data_volatility
    .dropna()
)

data_volatility.to_csv('../data/processed.csv')

display(data_volatility)


,Date,Close,Num_Workday,Test_Period_Ahead_Return,Single_Period_Ahead_Return,Test_Period_Ahead_Std
9,1988-01-15,252.289993,9,0.020532,-0.000674,0.000217
10,1988-01-18,252.119995,10,0.013446,-0.010114,0.000203
11,1988-01-19,249.570007,11,0.026365,-0.026846,0.000138
12,1988-01-20,242.869995,12,0.040845,0.002100,0.000139
13,1988-01-21,243.380005,13,0.039403,0.013847,0.000133
...,...,...,...,...,...,...
9301,2024-12-02,13261.080078,0,0.005105,0.000471,0.000022
9302,2024-12-03,13267.330078,1,0.000770,0.006071,0.000093
9303,2024-12-04,13347.879883,2,-0.034588,-0.001809,0.000093
9304,2024-12-05,13323.740234,3,-0.033673,0.002609,0.000110
